# Nemo Curator Pipeline Example

## NeMo Curator Introduction
The NeMo Curator is a Python library that consists of a collection of scalable data-mining modules for curating natural language processing (NLP) data for training large language models (LLMs). The modules within the NeMo Data Curator enable NLP researchers to mine high-quality text at scale from massive uncurated web corpora. 

NeMo Curator includes the following modules to perform data curation:
- Data download and Extraction
- Language identification and separation
- Text reformatting and cleaning
- Quality filtering
- Document-level deduplication
- Multilingual downstream-task decontamination
- Distributed Data Classification
- Personal identifiable information (PII) redaction

NeMo Curator team has perform ablation experiments using Common Crawl dataset to train a 357M GPT-style model to assess the effect of different curation stage on model performance. 

![alt text](./image/zeroshot_ablations.png)


## About this notebook


This notebook will use **Thai Wikipedia dataset** as example to demonstrate a typical data curation pipeline using NeMo Curator. After running through this script, user will be able to know how to use NDC to download wikipedia data, perform language separation using fasttext, perform GPU based exact deduplication and fuzzy deduplication and use CPU based heuristic filtering. 

Step description:
1. Download and extract data
2. Language detection and separation
3. GPU based deduplication
    1. Exact deduplication
    2. Fuzzy deduplication
4. Heuristic filtering

What is not included:
1. Customized downloading
2. Classifier filtering
3. Downstream-task decontamination
4. Distributed data classification with PyTorch models
5. Personal identifiable information (PII) redaction 



## Prerequisites

### System Requirements
Here is the hardware setting for this notebook

**GPU**: NVIDIA A10 24G. 

**CUDA & Nvidia Drivers**: CUDA 12.2 with Driver 535.154.05

**OS**: ubuntu 22.04

### Getting NeMo Framework Training Container
- Get access to the container via https://developer.nvidia.com/nemo-framework
- Set your docker credentials 
    ```bash
    docker login nvcr.io

    Username: $oauthtoken
    Password: <Your NGC Key>
- Get NeMo NeMo Framework Training Container
    ```bash
    docker pull nvcr.io/nvidia/nemo:dev


## 0. Env Setup

In [ ]:
!pip install jsonlines

In [ ]:
%env CUDA_VISIBLE_DEVICES 0

In [ ]:
import argparse
import os

from nemo_curator.utils.distributed_utils import get_client,get_num_workers
from nemo_curator.utils.file_utils import get_all_files_paths_under, separate_by_metadata
from nemo_curator.utils.distributed_utils import read_data,write_to_disk
from nemo_curator.datasets import DocumentDataset

import sys
import pandas as pd
import time
import cudf
import dask_cudf
import dask
import numpy as np
from dask.distributed import Client, LocalCluster
import jsonlines


In [ ]:
def pre_imports():
    import cudf 

def check_jsonl_file(file_dir):
    for file in os.listdir(file_dir):
        if 'jsonl' not in file:
            continue
        with open(os.path.join(file_dir,file), 'r', encoding='utf-8') as f:
            first_line = f.readline()
            print(first_line)
        break

def extract_lines_with_id(file_path,target_list):
    with jsonlines.open(file_path) as reader:
        for obj in reader:
            if obj.get('id') in target_list:
                yield obj

def get_base_dataset_file_name(download_folder):
    files = os.listdir(download_folder)
    for file in files:
        if file.startswith('thwiki') and file.endswith(''):
            return file

In [ ]:
cur_dir = os.getcwd()
print(cur_dir)
data_dir = f"{cur_dir}/workspace/"

## 1. Download
In this example, Thai wikipedia data will be downloaded.

Here is what happens when function `download_wikipedia()` is called:
1. Run `get_wikipedia_urls()` to obtain a list of urls to download .bz2 files for Thai wikipedia data. In this module, we use the base link and the language from user input to formulate a repo links for downloadable wikipedia .bz2 dump files. The formulated link will be `https://dumps.wikimedia.org/<language>wiki`. All the links will be stored in a .txt file. Argument for this function includes:
    - `dump_dates`: A date in the string format of 'YYYYMMDD'. It determines which wikipedia snapshot will be downloaded. If not specified, the `latest` snapshot will be downloaded
    - `language`: language code of the desired language in lower case. Default value is `en`

2. 
    Run `download_and_extract()` to download and extract contents based on the url list obtained from `get_wikipedia_urls`. User will need to define `downloader`, `extractor` and `iterator` for the dataset. 
    In this case, `WikipediaDownloader`,`WikipediaIterator` and `WikipediaExtractor` are used.
    - `WikipediaDownloader`: Downloads wikipedia dumps file to local folder.
    - `WikipediaIterator`: Extracts the .bz2 files and useful content from the base html content.
    -  `WikipediaExtractor`: Performs further task specific html content cleaning such as removing media files, removing references/tables etc. and finally yield pure text data which will be store in .jsonl format. 
    Please refer to `./NeMo-Curator/nemo_curator/download/wikipedia.py` for  detail implementation.
    
    Argument for this function includes:
    - `output_path`: Output path for downloaded and extracted dataset
    - `output_type`: Type of output file. Default is .jsonl. User might choose other types such as parquet. In this example, .jsonl will be used
    - `language`: See above
    - `dump_date`: See above
    - `raw_download_dir`: Output path for intermediate downloaded .bz2 file. If not specified, will be downloaded to `output_path`
    - `keep_raw_download`: Whether to keep downloaded .bz2 files after extraction. Default is not to keep.
    - `force_download`: Whether to restart downloading process if the target .bz2 files are detected under the `raw_download_dir` 
    - `url_limit`: Number of .bz2 files to be downloaded.

The resultant .jsonl for Thai wikipedia will contain the following keys:
1. text
2. title
3. id
4. url
5. language
6. source_id
7. file_name

In [ ]:
from nemo_curator.download import download_wikipedia

 Start a CPU based Dask cluster. Please modify `n_workers` and `memory_limit` according to your hardware specification. To process TH wikipedia data, it's advised to have `memory_limit` greater than 12GB

In [ ]:
cluster = LocalCluster(n_workers=10, processes=True, memory_limit='16GB')
client = Client(cluster)

Define parameters

In [ ]:
#Output
download_base_directory= os.path.join(data_dir,"wiki_downloads")
download_output_directory = os.path.join(download_base_directory,"data")

#Relevant parameters
language = 'th'
url_limit = 1

Download TH wikipedia data

In [ ]:
res = download_wikipedia(download_output_directory,
                   language=language, 
                   url_limit=url_limit).df.compute()

**[Optional]** Verify result

In [ ]:
# List all the file in the output directory.
# ! ls {download_output_directory}

# Please replace your dataset file name accordingly.
# ! wc -l  {download_output_directory}/{YOUR DATASET FILE NAME}.jsonl

In [ ]:
check_jsonl_file(download_output_directory)

In [ ]:
!rm -r {download_output_directory}/downloads

**[Optional]** Close the Dask cluster.You might encounter error such as `Caught signal 11`.It's OK, just rerun the cell again.

In [ ]:
# client.cluster.close()
# client.shutdown()

## 2.Language separation and unicode fixing

In this section, we will be using a language classification model by fasttext to separate the TH wikipedia dataset based on the document major languages, and we will also fix the unicode in the documents. Detailed steps are:

1. Download fasttext model for text language detection
2. Construct a filter which uses the downloaded fasttext model to produce a language label to each document. 
3. Separate each document by the language label. This will create sub-folders for each languages under the output path and the documents under the same language will be output to a .jsonl file in the corresponding sub-folder.
4. Load .jsonl file in the folder of desirable language. In this example, `TH` folder will be loaded.
5. Apply `UnicodeReformatter` to the data and output the result in .jsonl format. 



In [ ]:
from nemo_curator import ScoreFilter,Modify
from nemo_curator.filters import FastTextLangId
from nemo_curator.modifiers import UnicodeReformatter

**[Optional]** Start a cpu based Dask cluster.

In [ ]:
# cluster = LocalCluster(n_workers=10, processes=True, memory_limit='16GB')
# client = Client(cluster)

Define parameters

In [ ]:
# Input path
multilingual_data_path = download_output_directory

# Output path
language_base_output_path = os.path.join(data_dir,"language_sep")
language_data_output_path = os.path.join(language_base_output_path,"data")
language_separated_output_path = os.path.join(language_data_output_path,"language")
lang_sep_cleaned_data_output_path = os.path.join(language_data_output_path,"cleaned")

# Fasttext model path
model_path = language_base_output_path

# Define desired language
target_language = "TH"

# Define key in output .jsonl files to store the language information
language_field = "language"

Download fasttext model

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin -P {model_path}

Apply fasttext model to separate documents by their languages

In [ ]:
t0 = time.time()

# Load dataset 
multilingual_dataset = DocumentDataset.read_json(multilingual_data_path,add_filename=True)

#Define Language separation pipeline
lang_filter = FastTextLangId(os.path.join(model_path,'lid.176.bin'))
language_id_pipeline = ScoreFilter(lang_filter, score_field=language_field, score_type='object')
filtered_dataset = language_id_pipeline(multilingual_dataset)

# The language separation pipeline will produce a result looks like ['EN',0.96873], we only want to keep the 'EN' label and drop the detailed classifier score
filtered_dataset.df[language_field] = filtered_dataset.df[language_field].apply(lambda score: score[1],meta = (language_field, 'object'))

# Split the dataset to corresponding language sub-folders
language_stats = separate_by_metadata(filtered_dataset.df, language_separated_output_path, metadata_field=language_field).compute()

print(f"Time taken for splitting language:{time.time()-t0}")

Load `UnicodeReformatter` to reformat any unicode appeared in the desired language dataset

In [ ]:
t0 = time.time()

# Read the language specific data and fix the unicode in it
lang_data_path = os.path.join(language_separated_output_path, target_language)
lang_data = DocumentDataset.read_json(lang_data_path,add_filename=True)

cleaner = Modify(UnicodeReformatter())
cleaned_data = cleaner(lang_data)

# Write the cleaned_data
cleaned_data.to_json(lang_sep_cleaned_data_output_path, write_to_filename=True)

print(f"Time taken for fixing unicode:{time.time()-t0}")

**[Optional]** Verify the result. We can see that some documents has been removed from TH wikipedia dataset since the number of lines in this output file is less than the original file 

In [ ]:
# List all the file in the output directory.
# ! ls {lang_sep_cleaned_data_output_path}

# Please replace your dataset file name accordingly.
# ! wc -l  {lang_sep_cleaned_data_output_path}/{YOUR DATASET FILE NAME}.jsonl

Furthur verify by loading documents that has been identified as other language, such as 'EN'. We can see from output that the removed document is indeed in English and contains very little or even no Thai.

In [ ]:
check_jsonl_file(os.path.join(language_separated_output_path,'EN'))

**[Optional]** Close the Dask cluster.

In [ ]:
# client.cluster.close()
# client.shutdown()

## 3.Add ID
TH wikipedia data do have `id` field, but the `id` field contains number only. It will be better if we unified the `id` field and transform it to the format of `<prefix>_<id>`. In this way, when handling multiple dataset, we will be able to know which document from which dataset has been removed. This `id` will be useful when we are running deduplication and heuristic filtering. The function we will be using is `AddID()`. Arguments for this function include:
- `id_field`: fields will be added to input .json file. If the key already exists in the .jsonl, it's value will be replaced.
- `id_prefix`: prefix used in ID. Default is 'doc_id'
- `start_index`: starting index in ID. Default is None. When set to None, an unordered ID scheme will be used for fast calculation. In this notebook, it's set to 0 for easier reference.

In [ ]:
from nemo_curator import AddId

**[Optional]** If there is no running Dask cluster, start CPU based Dask cluster.

In [ ]:
# cluster = LocalCluster(n_workers=10, processes=True, memory_limit='16GB')
# client = Client(cluster)

Define relevant parameters

In [ ]:
#Input
add_id_input_data_dir = lang_sep_cleaned_data_output_path

#Output
added_id_output_path = os.path.join(data_dir,"add_id/cleaned")

#Format of output ID will be <prefix>_<id>, Define prefix here
add_ID_id_prefix="TH_wiki"

Adding ID to dataset

In [ ]:
t0 = time.time()
# Read input files
dataset = DocumentDataset.read_json(add_id_input_data_dir,add_filename=True)

# Run AddID() on the input dataset
add_id = AddId(id_field='id',id_prefix=add_ID_id_prefix,start_index=0)
id_dataset = add_id(dataset)

#Output files
id_dataset.to_json(added_id_output_path, write_to_filename=True)

print(f"Time taken for add ID:{time.time()-t0}")

Verify the result. From the output, we can see that the `id` value has been changed to `TH_wiki-0000000000` 

In [ ]:
check_jsonl_file(added_id_output_path)

Close Dask cluster. This cell needs to be run as we are starting a new GPU Dask cluster in the following task

In [ ]:
client.cluster.close()
client.shutdown()

## 4.Exact Dedplication

In exact deduplication, the document text is hashed into unique string using certain hashing algorithm, such as 'md5'. The documents with exact hashed values are having identical text. We will output the `ID` of duplicated documents for removal later. The function used is `ExactDuplicates()`. Arguments for this function include:
- `id_field`: Key in input file for identifying document ID
- `text_field`: Key in input file which contains document text.
- `hash_method`: Hashing algorithm used. Default is `md5`
- `cache_dir`: If specified, the duplicated document IDs will be output to the `cache_dir`. Otherwise, the IDs will not be saved

Also, we are going to use GPU dask cluster to accelerate computation for deduplication (both exact and fuzzy)


In [ ]:
from nemo_curator.modules import ExactDuplicates

Start a GPU based Dask cluster. Since GPU based Dask cluster involves setting several arguments, we will use the `get_client()` wrapper function to quickly set up. 

In [ ]:
client = get_client(cluster_type = 'gpu', set_torch_to_use_rmm=False)
print(f"Number of dask worker:{get_num_workers(client)}")
client.run(pre_imports)

If you encounter the following error
`get_client() missing 1 required positional argument: 'args'`:

This is probably because the `nemo_curator` library is not updated to the newer version. Please run the following line in the terminal, following instruction in our [GitHub](https://github.com/nicoleeeluo/NeMo-Curator/tree/main) repo, and restart the notebook. Intermediate result of the previous section has been saved to local, you can start from this section after updating.

In [ ]:
#pip install --extra-index-url https://pypi.nvidia.com ".[cuda12x]"

Define parameters

In [ ]:
#Input
exact_dedup_input_dataset_dir = added_id_output_path

#Output
exact_dedup_base_output_path = os.path.join(data_dir,"exact_dedup")
exact_dedup_log_dir = os.path.join(exact_dedup_base_output_path,'log')
exact_dedup_output_dir = os.path.join(exact_dedup_base_output_path,'data')

#Parameters for ExactDuplicates()
exact_dedup_dataset_id_field = "id"
exact_dedup_dataset_text_field = "text" 


In [ ]:
!mkdir -p {exact_dedup_log_dir}
!mkdir -p {exact_dedup_output_dir}

Apply exact deduplication

In [ ]:
t0 = time.time()
# Read input dataset
input_dataset = DocumentDataset.read_json(exact_dedup_input_dataset_dir, backend='cudf')

#Run exact deduplication to the input
exact_dup = ExactDuplicates(
    logger=exact_dedup_log_dir,
    id_field=exact_dedup_dataset_id_field,
    text_field=exact_dedup_dataset_text_field,
    hash_method="md5",
    cache_dir=exact_dedup_output_dir #Duplicated document ID list is output to the cache_dir
)
duplicates = exact_dup(dataset=input_dataset)

print(f"Number of exact duplicated file:{len(duplicates)}")

print(f"Time taken for exact duplicate:{time.time()-t0}")

**[Optional]** Verify the output duplicated ID. We can group by the `_hashes` to get the list of duplicated documents having the same _hashes and use `extract_lines_with_id()` to verify that those documents are indeed exact duplicates. Please note that the `id` might changes, therefore, please replace the `target_list` when necessary

In [ ]:
exact_dedup_res = pd.read_parquet(os.path.join(exact_dedup_output_dir,"_exact_duplicates.parquet"))
print(f"Number of exact duplicated document:{len(exact_dedup_res)}")
exact_dedup_res.head()

In [ ]:
exact_dedup_res.groupby('_hashes')['id'].agg(lambda x: ' '.join(x)).reset_index().head()

Using the duplicated id shown above, check the content to see if it's exact duplicates

In [ ]:
# target_list = ['TH_wiki-0000157216', 'TH_wiki-0000066307']
# for line in extract_lines_with_id(os.path.join(exact_dedup_input_dataset_dir,'{YOUR DATASET FILE NAME}'),target_list):
#     print(line)

**[Optional]** You might choose to close Dask cluster here

In [ ]:
# client.cluster.close()
# client.shutdown()

## 5. Fuzzy Deduplication
Fuzzy deduplication involves 5 intermediate steps to generate duplicates. Refer to https://docs.nvidia.com/nemo-framework/user-guide/latest/datacuration/gpudeduplication.html for details

Fuzzy deduplication in this example is a GPU implementation of MinhashLSH algorithm. This algorithm measures similarity based on statistics but not semantic meanings of text. There are a few concepts to be introduced before heading into fuzzy deduplication.
1. Jaccard similarity: Jaccard similarity is often used as a metric to calculate the similarity between two sets. It's calculated by dividing the number of common elements in the two sets (Intersection) by the number of total unique elements in the two sets (Union). In the case of text documents, we transform a document into a set of n-grams. If two documents share a large amount of n-grams, most likely the documents are similar. 

    ![alt text](./image/jaccard.png )

2. Complexity of the problem: To find all the similar document pairs in a dataset, we need to compute pair-wise Jaccard similarity across the dataset. Hence, making the complexity $O(N^2)$

The MinhashLSH algorithm is a technique for quickly estimating the similarity between sets, such as the similarity between documents represented as sets of shingles (n-grams). It's able to find out Jaccard similar pair in the corpus but in a much computational efficient way. This algorithm has following steps in a high-level:
1. Compute minhash for each document
2. Run Locality Sensitive Hashing (LSH) based on the minhash which further assign buckets to each document. Each documents will be assigned to multiple buckets. Documents within the same bucket are deemed to be similar.
3. Run pair-wise Jaccard similarity within each buckets to remove false positive cases within the buckets
4. Based on the Jaccard similarity, transform the similarity matrix to a graph ans run connected component algorithm. For a group of connected components in the graph, they are the final similar document groups and the IDs within each groups will be output for duplicate removal.
More detailed explanation please refer to https://docs.nvidia.com/nemo-framework/user-guide/latest/datacuration/cpudeduplication.html.

For implementation of MinhahsLSH on GPU, there are 5 steps:
1. Minhash computation
2. Bucket computation
3. Jaccard shuffle for load balancing in a distributed system
4. Jaccard similarity computation
5. Connected component 

In this section, we will firstly provide examples to each sub-steps for users to have a better understanding on what is going on under the hood. At the last sub section, we will provide example for the fuzzy deduplication wrapper.

**If there is not running Dask cluster, start a GPU Dask cluster here**

In [ ]:
# client = get_client(cluster_type = 'gpu', set_torch_to_use_rmm=False)
# print(f"Number of dask worker:{get_num_workers(client)}")
# client.run(pre_imports)

### 5.1 Minhash

Run `MinHash()` for this section. The output of a minhash is a parquet file which contains document ID and hashed value which is an array contains 260 32-bit integer data. To obtain such hashed values we need to go through the following steps:
1. Generate a set of n-gram components of a document. For example, doc = `Nemo Curator is a data curation tool`, a 3-gram set of this document will be `['Nemo Curator is','Curator is a','is a data','a data curation','data curation tool']`
2. Hashed each n-gram into numerical values
3. Generate a random hash function $H_1()$ which will hash each numeric n-gram into a 32-bit integer and take the minimum integer to use as minhash value for $H_1()$
4. Repeat step 2 and 3 with hash function $H_x()$ until desired minhash length is reached. Minhash value of each iteration will be append together to form the final minhash array. 

Arguments include:
- `seed`:Random seed used for initializing the hash functions used to compute the MinHashes. It's advised to keep this value the same for different experiment for reproducibility
- `num_hashes`:Length of each minhash array. Default is 260. Longer minhash length will have better estimate of actual Jaccard similarity, but require more computational power
- `char_ngrams`:n-gram length
- `use_64bit_hash`:Whether to use 64bit or 32bit hash function
- `id_field`: Key in input file for identifying document ID
- `text_field`: Key in input file which contains document text.
- `cache_dir`: If specified, the intermediate result will be output to the `cache_dir`. 



In [ ]:
from nemo_curator import MinHash

Define parameters

In [ ]:
#Input
minhash_data_path = added_id_output_path
#Output
minhash_base_output_path = os.path.join(data_dir,"fuzzy/minhash")
minhash_log_dir = os.path.join(minhash_base_output_path,'log')
minhash_output_dir = os.path.join(minhash_base_output_path,'data')
#Specify dataset name
dataset_name = 'TH_wikipedia'

#Relevant parameters
minhash_id_field = 'id'
minhash_text_field = 'text'
seed = 10
minhash_length = 260
char_ngram = 5
use_64bit_hash = False
files_per_partition = 2

!mkdir -p {minhash_log_dir}
!mkdir -p {minhash_output_dir}

Run MinHash

In [ ]:
t0 = time.time()
print(f"Computing minhashes for {minhash_data_path}")

# Load data. Only the [minhash_id_field, text_field] columns are needed
files = get_all_files_paths_under(root=minhash_data_path, recurse_subdirectories=False)
files = [f for f in files if f.endswith(".jsonl")]
df = read_data(
    files,
    file_type="jsonl",
    backend="cudf",
    files_per_partition=files_per_partition,
    add_filename=False,
)[[minhash_id_field, minhash_text_field]]

# Run MinHash() on input data
minhasher = MinHash(
    seed=seed,
    num_hashes=minhash_length,
    char_ngrams=char_ngram,
    use_64bit_hash=use_64bit_hash,
    logger=minhash_log_dir,
    id_field=minhash_id_field,
    text_field=minhash_text_field,
    cache_dir=minhash_output_dir
)
res = minhasher(DocumentDataset(df)).df

print(f"Time taken for MinHash:{time.time()-t0}")

**[Optional]** Verify result

In [ ]:
# minhash_res = pd.read_parquet(os.path.join(minhash_output_dir, "_minhashes.parquet"))
# minhash_res.head()

### 5.2 LSH
`LSH()` implements LSH algorithm which includes the following steps:
1. Divide the minhash array into `X` different portions. 
2. For each portions, hash the minhash values into buckets. One document will be assigned to `X` buckets.
3. Documents within the same bucket will be deemed similar. Since every document will be assigned `X` buckets and as long as two documents share 1 or more buckets they are deemed similar, the result of LSH will have more false positive as compared to false negative. The false positive cases will be filtered in following modules, namely jaccard compute.

Arguments include:
- `minhash_length`:Length of minhash signature. Must be consistent with `MinHash()`
- `num_buckets`: Number of buckets
- `buckets_per_shuffle`: Number of buckets to shuffle concurrently
- `id_field`: Key in input file for identifying document ID
- `minhash_field`: Key in input file for identifying document MinHash signature 
- `cache_dir`:If specified, the intermediate result will be output to the `cache_dir`.



In [ ]:
from nemo_curator import LSH
from nemo_curator.utils.fuzzy_dedup_utils.id_mapping import \
    convert_str_id_to_int

Define parameters

In [ ]:
#Input
lsh_input_data_path = minhash_output_dir

#Output
lsh_base_output_path = os.path.join(data_dir,"fuzzy/lsh")
lsh_log_dir = os.path.join(lsh_base_output_path,'log')
lsh_output_dir = os.path.join(lsh_base_output_path,'data')

#Relevant parameters
lsh_id_field = 'id'
minhash_field = '_minhash_signature'
minhash_length=260
num_bands=20
buckets_per_shuffle=1

!mkdir -p {lsh_log_dir}
!mkdir -p {lsh_output_dir}

Run LSH

In [ ]:
t0 = time.time()

#Load MinHash output
df = dask_cudf.read_parquet(lsh_input_data_path, blocksize="2GB", aggregate_files=True, backend = "cudf")
df = df.map_partitions(
    convert_str_id_to_int,
    id_column=lsh_id_field,
    meta=cudf.DataFrame(
        {minhash_field: [[1, 2, 3]], "doc_id": [1], "dataset_id": np.uint32(1)}
    ),
)

#Run LSH()
lsh = LSH(
    cache_dir=lsh_output_dir,
    num_hashes=minhash_length,
    num_buckets=num_bands,
    buckets_per_shuffle=buckets_per_shuffle,
    id_fields=["dataset_id", "doc_id"],
    minhash_field=minhash_field,
    logger=lsh_log_dir,
)
res = lsh(DocumentDataset(df))

t1 = time.time()
print(f"Time taken for LSH:{time.time()-t0}")

**[Optional]** Verify result

In [ ]:
# lsh_res = pd.read_parquet(os.path.join(lsh_output_dir, "_buckets.parquet"))
# lsh_res.head()

### 5.3 Jaccard Shuffle
In this section, we will be using `_MapBucket()` and `_Shuffle()`.

For `_MapBucket()`, it is designed to take input text data in jsonl format and bucket information which is output of LSH, map the documents to their respective buckets, and write the resulting DataFrame containing the anchor documents and their associated bucket information to a parquet file. Arguments include:
- `id_field`: Key in input .jsonl file for identifying document ID
- `text_field`: Key in input .jsonl file which contains document text.
- `bucket_field`: Key in input _buckets.parquet which contains `bucket_id`.
- `num_anchors`: Number of anchors (document in the same buckets) to be output


For `_Shuffle()`, it perform a shuffling operation on the documents based on their bucket assignments, output in .parquet format. This shuffling operation is a crucial step in the deduplication process, as it helps distribute similar documents across different partitions or workers, enabling efficient parallel processing and deduplication in subsequent steps. Arguments include:
- `id_fields`: Columns in `_buckets.parquet` that maps to original `id` in .jsonl data file. In this example, it is `["dataset_id", "doc_id"]`
- `text_field`: Key in input .jsonl file which contains document text.
- `int_to_str_id`:  Key in input .jsonl file for identifying document ID


In [ ]:
from nemo_curator.utils.fuzzy_dedup_utils.io_utils import (
    get_bucket_ddf_from_parquet_path,
    get_text_ddf_from_json_path_with_blocksize,
)
from nemo_curator.modules.fuzzy_dedup import _MapBuckets,_Shuffle

Define parameters

In [ ]:
#Input
input_data_paths = [minhash_data_path]
input_bucket_path = lsh_output_dir

#Output
jaccard_shuffle_base_output_path = os.path.join(data_dir,"fuzzy/jaccard_shuffle")
output_anchor_docs_with_bk_path = os.path.join(jaccard_shuffle_base_output_path, "anchor_docs_with_bk.parquet")
input_anchor_docs_with_bk_dir = output_anchor_docs_with_bk_path
jaccard_shuffle_log_path = os.path.join(jaccard_shuffle_base_output_path,"log")
output_shuffled_docs_path = os.path.join(jaccard_shuffle_base_output_path, "shuffled_docs.parquet")

#Relevant parameters for _MapBucket()
text_ddf_blocksize = 256
bucket_mapping_ddf_blocksize = 256
num_files = None
shuffle_type ='tasks'
input_bucket_field = '_bucket_id'
input_id_field = 'id'
input_text_field = 'text'

#Relevant parameters for _Shuffle()
shuffle_id_fields=["dataset_id", "doc_id"]
int_to_str_id='id'

!mkdir -p {jaccard_shuffle_base_output_path}
!mkdir -p {jaccard_shuffle_log_path}

Run Jaccard map bucket

In [ ]:
t0 = time.time()
num_workers = get_num_workers(client)

# Read .jsonl input data
ddf_text = get_text_ddf_from_json_path_with_blocksize(
    input_data_paths=input_data_paths,
    num_files=num_files,
    blocksize=text_ddf_blocksize,
    id_column=input_id_field,
    text_column=input_text_field,
)
# Read "_buckets.parquet"
ddf_bk = get_bucket_ddf_from_parquet_path(input_bucket_path=input_bucket_path, num_workers=num_workers)

#Run _MapBuckets()
map_buckets = _MapBuckets(id_fields=shuffle_id_fields, bucket_field=input_bucket_field, logger=jaccard_shuffle_log_path)
ddf_anchor_docs_with_bk = map_buckets.map_buckets_with_anchors(documents_df=ddf_text, buckets_df=ddf_bk, shuffle_type=shuffle_type)

#Write to disk
ddf_anchor_docs_with_bk.to_parquet(output_anchor_docs_with_bk_path, write_index=False)

print(f"Time taken for Bucket Mapping:{time.time()-t0} s")

**[Optional]** Verify result

In [ ]:
# map_bucket_res = pd.read_parquet(output_anchor_docs_with_bk_path)
# map_bucket_res.head()

**[Optional]** Remove previous Jaccard Shuffle results. Run only when there are files under the Jaccard Shuffle output path

In [ ]:
#!rm -r {output_shuffled_docs_path}

Run Jaccard Shuffle

In [ ]:
t0 = time.time()

#Run _Shuffle() on results of _MapBucket()
shuffle = _Shuffle(
    id_fields=shuffle_id_fields,
    text_field=input_text_field,
    int_to_str_id=int_to_str_id,
    logger=jaccard_shuffle_log_path
)
shuffle.shuffle_docs_on_buckets(
    documents_df=ddf_text,
    bucket_w_anchors_path=input_anchor_docs_with_bk_dir,
    output_shuffled_docs_path=output_shuffled_docs_path,
    bucket_mapping_df_blocksize=bucket_mapping_ddf_blocksize,
#     parts_per_worker=1,
#     bucket_parts_per_worker=8,
    partition_on="_output_partition_id",
)

print(f"Time taken for Jaccard Shuffle = {time.time()-t0} s")

**[Optional]** Verify result

In [ ]:
# jaccard_shuffle_res = pd.read_parquet(os.path.join(output_shuffled_docs_path,"_output_partition_id=0/batch_1_1.parquet"))
# jaccard_shuffle_res.head()

### 5.4 Jaccard Compute
We will be using `JaccardSimilarity()`.This is to computes the Jaccard similarity between document pairs. Result is a parquet dataset consisting of document id pair along with their Jaccard similarity score. To compute Jaccard similarity between two documents, we first convert the document into sets of n-grams and then compute the Jaccard similarity of the two sets.

Arguments include:
- `id_field`: Column in input .parquet file identifying document ID
- `text_field`: Column in input .parquet file identifying document text
- `anchor_id_fields`: Column in input .parquet file identifying anchors. This can be generated by specifying number of anchor used in `_MapBucket` whose default value is 2
- `ngram_width`: n-gram used

In [ ]:
from nemo_curator.modules.fuzzy_dedup import JaccardSimilarity

Define parameters

In [ ]:
#Input
shuffled_docs_path = output_shuffled_docs_path

#Output
jaccard_compute_base_output_path = os.path.join(data_dir,"fuzzy/jaccard_compute")
jaccard_compute_output_results_path = os.path.join(jaccard_compute_base_output_path, "jaccard_similarity_results.parquet")

#Relevant parameters
input_id_field = 'id'
input_text_field = 'text'
ngram_size = 5
num_anchors = 2

!mkdir -p {jaccard_compute_base_output_path}

Run Jaccard Compute

In [ ]:
# enable_spilling()
# client.run(enable_spilling)

print("Running jaccard compute script", flush=True)
t0 = time.time()

jaccard = JaccardSimilarity(
    id_field=input_id_field,
    text_field=input_text_field,
    anchor_id_fields=[f"anchor_{i}_{input_id_field}" for i in range(num_anchors)],
    ngram_width=ngram_size,
)

#Load and run Jaccard compute
result_df = jaccard.jaccard_compute(shuffled_docs_path)

result_df.to_parquet(jaccard_compute_output_results_path, write_index=False, write_metadata_file=False)

print(f"Time taken for Jaccard Computing: {time.time()-t0}")

**[Optional]** Verify output. You might see that there are repeated `id_x` and `id_y` pairs. This is expected as a pair of similar documents is likely to share numerous same buckets.

In [ ]:
# jaccard_compute_res = pd.read_parquet(jaccard_compute_output_results_path)
# jaccard_compute_res.head()

### 5.5 Connected Components
This section uses `ConnectedComponents()`.This section takes a dataset consisting of document pairs and their corresponding jaccard similarity to construct a non-directed graph. A edge will be form between documents whose Jaccard similarity is higher than the threshold (0.8 in this example). It will then identify the connected components in this graph. Documents within the same connected components are deemed duplicated

Arguments include:
- `cache_dir`:Output path for intermediate results
- `jaccard_pairs_path`:Input path for `jaccard_similarity_results.parquet`
- `id_column`:prefix of ID column in `jaccard_similarity_results.parquet`
- `jaccard_threshold`:Threshold to determine if an edge exists between two documents

In [ ]:
from nemo_curator.modules.fuzzy_dedup import ConnectedComponents

Define parameters

In [ ]:
#Input
jaccard_pairs_path = jaccard_compute_output_results_path

#Output
connected_component_base_output_path = os.path.join(data_dir,"fuzzy/cc")
connected_component_output_path = os.path.join(connected_component_base_output_path, "connected_components.parquet")
connected_component_cache_dir = os.path.join(connected_component_base_output_path, "cache")

#Relevant parameters
input_id_field = 'id'
jaccard_threshold = 0.8

!mkdir -p {connected_component_base_output_path}

Run Connected Component

In [ ]:
t0 = time.time()
    
components_stage = ConnectedComponents(
    cache_dir=connected_component_cache_dir,
    jaccard_pairs_path=jaccard_pairs_path,
    id_column=input_id_field,
    convert_str_ids=True,
    jaccard_threshold=jaccard_threshold,
)

#Load and run connected component
components_stage.cc_workflow(output_path=connected_component_output_path)
print(f"Time taken for Connected Component: {time.time()-t0} s")

**[Optional]** Run the following cells to verify the result of `Connected Components`

In [ ]:
# cc_compute_res = pd.read_parquet(connected_component_output_path)
# cc_compute_res.head()

Let's check if the output fuzzy duplicated documents within the same group are similar. Please note that the `group` id in your output might be different from the notebook output.

In [ ]:
# cc_compute_res['doc_id'] = cc_compute_res['doc_id'].astype(str)
# cc_compute_res.groupby('group')['doc_id'].agg(lambda x: ', '.join(x)).reset_index()

Change the `group` number if necessary. By running the code below, we can obtain a list of near duplicated documents.

In [ ]:
#Repalce ??? with the group number you want to check
# cc_compute_res[cc_compute_res['group']==???].head()

Print the text of near duplicated document. Please replace the `id` if necessary, `id` should be in the format of `<dataset_id>_<doc_id>`

In [73]:
# Repalce 'ID1' and 'ID2' with IDs you want to check
# The output is an example of fuzzy duplicates 

# jaccard_shuffle_res[jaccard_shuffle_res['id'].isin(['ID1','ID2'])]['text'].unique()

array(['ประเทศสวิตเซอร์แลนด์ ได้เข้าร่วมแข่งขันกีฬาโอลิมปิกเยาวชนฤดูหนาว ครั้งที่ 3 ค.ศ. 2020 (พ.ศ. 2563) ณ เมืองโลซาน ประเทศสวิตเซอร์แลนด์ ระหว่างวันที่ 9 - 22 มกราคม พ.ศ. 2563 คณะกรรมการโอลิมปิกแห่งชาติสวิตเซอร์แลนด์ได้ส่งทีมนักกีฬาเข้าแข่งขันทั้งหมด 56 คน แบ่งเป็นเป็นชาย 32 คนและหญิง 56 คน เข้าร่วมการแข่งขันใน 15 ชนิดกีฬา\n\nจำนวนผู้เข้าแข่งขัน\n\nผลการแข่งขัน\n\nสเกตลีลา\n\nสเกตความเร็ว\n\nสเกตความเร็วระยะสั้น\n\nฮอกกี้น้ำแข็ง\n\nเคอร์ลิง\n\nสกีลงเขา\n\nสกีข้ามทุ่ง\n\nสกีกระโดดไกล\n\nสกีนอร์ดิกผสม\n\nสกีลีลา\n\nสกีปีนเขา\n\nสโนว์บอร์ด\n\nทวิกีฬาฤดูหนาว\n\nบอบสเล\n\nสเกเลตัน\n\nอ้างอิง\n\nแหล่งข้อมูลอื่น \n เว็บไซต์อย่างเป็นทางการ \n\nประเทศสวิตเซอร์แลนด์ในโอลิมปิกเยาวชน\nประเทศที่เข้าร่วมแข่งขันโอลิมปิกเยาวชนฤดูหนาว 2020',
       'ประเทศบัลแกเรีย ได้เข้าร่วมแข่งขันกีฬาโอลิมปิกเยาวชนฤดูหนาว ครั้งที่ 3 ค.ศ. 2020 (พ.ศ. 2563) ณ เมืองโลซาน ประเทศสวิตเซอร์แลนด์ ระหว่างวันที่ 9 - 22 มกราคม พ.ศ. 2563 คณะกรรมการโอลิมปิกแห่งชาติบัลแกเรียได้ส่งทีมนักกีฬาเข้าแข่งขันทั้งหมด 18 คน แบ่งเป็นเป็นชา

Below is the English translation of the output above. We can see that the two documents are indeed very similar to each other.
- `Text 1`:
```
Switzerland participated in the 3rd Youth Olympic Winter Games in 2020 (B.E. 2563) in Lausanne, Switzerland from January 9 - 22, 2563. The Swiss Olympic Committee sent a total of 56 athletes, consisting of 32 men and 56 women, to compete in 15 sports.
Number of Competitors:
Competition Results:
Figure Skating
Speed Skating
Short Track Speed Skating
Ice Hockey
Curling
Alpine Skiing
Cross-Country Skiing
Ski Jumping
Nordic Combined
Freestyle Skiing
Ski Mountaineering
Snowboard
Biathlon
Bobsleigh
Skeleton
References:
Other Resources:
Official Website
Switzerland at the Youth Olympics
Countries at the 2020 Youth Winter Olympics
```
- `Text 2`:
```
Bulgaria participated in the 3rd Youth Olympic Winter Games in 2020 (B.E. 2563) in Lausanne, Switzerland from January 9 - 22, 2563. The Bulgarian Olympic Committee sent a total of 18 athletes, consisting of 11 men and 7 women, to compete in 8 sports.
Number of Competitors:
Competition Results:
Figure Skating
Speed Skating
Short Track Speed Skating
Ice Hockey
Curling
Alpine Skiing
Cross-Country Skiing
Ski Jumping
Nordic Combined
Freestyle Skiing
Ski Mountaineering
Snowboard
Biathlon
Luge
Bobsleigh
Skeleton
References:
Other Resources:
Official Website
Bulgaria at the Youth Olympics
Countries at the 2020 Youth Winter Olympics
```


### 5.6 Fuzzy deduplication wrapper

In [ ]:
from nemo_curator import FuzzyDuplicates, FuzzyDuplicatesConfig

In [ ]:
#Input
fuzzy_dedup_data_path = added_id_output_path
#Output
fuzzy_dedup_base_output_path = os.path.join(data_dir,"fuzzy_wrapper")
fuzzy_dedup_log_dir = os.path.join(fuzzy_dedup_base_output_path,'log')
fuzzy_dedup_cache_dir = os.path.join(fuzzy_dedup_base_output_path,'cache')
fuzzy_dedup_output_dir = os.path.join(fuzzy_dedup_base_output_path,'data')
#Specify dataset name
dataset_name = 'TH_wikipedia'

#Relevant parameters
id_field = 'id'
text_field = 'text'
filetype = "parquet"

!mkdir -p {fuzzy_dedup_base_output_path}
!mkdir -p {fuzzy_dedup_log_dir}
!mkdir -p {fuzzy_dedup_cache_dir}
!mkdir -p {fuzzy_dedup_output_dir}

**[Optional]** If the cache folder is not empty, please CLEAR the folder before proceeding

In [ ]:
#!rm -r {fuzzy_dedup_cache_dir}

In [ ]:
with dask.config.set({"dataframe.backend": 'cudf'}):
        
        t0 = time.time()
        
        input_dataset = DocumentDataset.read_json(fuzzy_dedup_data_path, backend='cudf')

        fuzzy_dedup_config = FuzzyDuplicatesConfig(
            cache_dir=fuzzy_dedup_cache_dir,
            id_field=id_field,
            text_field=text_field,
            seed=seed, #Use the seed set in Minhash section for consistency
            char_ngrams=5,
            num_buckets=20,
            hashes_per_bucket=13,
            use_64_bit_hash=False,
            buckets_per_shuffle=5,
            false_positive_check=True,
            num_anchors=2,
            jaccard_threshold=0.8,
        )
        fuzzy_dup = FuzzyDuplicates(logger=fuzzy_dedup_log_dir, config=fuzzy_dedup_config)
        duplicates = fuzzy_dup(dataset=input_dataset)
        
        duplicates.to_parquet(fuzzy_dedup_output_dir, write_to_filename=False)
       
        print(f"Time taken for Connected Component: {time.time()-t0} s")


In [ ]:
fuzzy_dedup_res = pd.read_parquet(fuzzy_dedup_output_dir)
fuzzy_dedup_res.head()

## 6. Remove duplicates

Now we have duplicated document IDs output by both exact deduplication and fuzzy deduplication. We will run this section to remove those documents. This is done be loading the output .parquet files and the unicode fixed input dataset in .jsonl as DataFrame. Then use DataFrame operation to remove the duplicated documents.

Define parameters

In [ ]:
#Input
dataset_dir = added_id_output_path

#Output
dudped_output_dir = os.path.join(data_dir,"remove_duplicate/result.parquet")

#Relevant parameters
input_id_field = 'id'
id_prefix = add_ID_id_prefix

!mkdir -p {dudped_output_dir}

We will first process the result of exact deduplication. Since result of exact deduplication contains original ID used in input dataset, it is more straightforward to deal with.

In [ ]:
#Load .jsonl dataset
input_dataset = DocumentDataset.read_json(dataset_dir, backend='cudf')

#Load exact deduplicate result and extract list of duplicated document ID
exact_duplicates = DocumentDataset.read_parquet(os.path.join(exact_dedup_output_dir,"_exact_duplicates.parquet"), backend='cudf')
exact_docs_to_remove = exact_duplicates.df.map_partitions(
    lambda x: x[x._hashes.duplicated(keep="first")]
)

#Remove the duplicated document from input dataset
result = input_dataset.df[
    ~input_dataset.df[input_id_field].isin(exact_docs_to_remove[input_id_field].compute())
]

For result of fuzzy deduplication, we need to first reconstructed document ID by combining `dataset_id` and `doc_id`, then use the reconstructed `ID` for removal

**[Optional]** Uncomment the cell to use result from step by step fuzzy deduplication

In [ ]:
# #List of id_prefix used in Add ID
# base_ids = [id_prefix]

# #Obtain a mapping between `dataset_id` and `id_prefix`
# df = cudf.DataFrame()
# df['base_id'] = [base_id for base_id in base_ids]
# df['dataset_id'] = df['base_id'].hash_values()
# df_pd = df.to_pandas()
# mapping = {
#       hashed_id: base_id
#       for base_id, hashed_id in zip(df_pd['base_id'], df_pd['dataset_id'])
# }

# #Load result of fuzzy deduplication 
# fuzzy_duplicates = pd.read_parquet(connected_component_output_path)
# #Reconstruct the original document ID
# fuzzy_duplicates['id']=fuzzy_duplicates.apply(lambda x: f"{mapping[x['dataset_id']]}-{x['doc_id']:010d}", axis=1)

# #Generate list of near duplicate document ID
# fuzzy_docs_to_remove = fuzzy_duplicates.drop_duplicates(subset=['group'], keep='first')

In [ ]:
#Loads result from fuzzy dedup wrapper
fuzzy_duplicates = pd.read_parquet(fuzzy_dedup_output_dir)

#Generate list of near duplicate document ID
fuzzy_docs_to_remove = fuzzy_duplicates[fuzzy_duplicates.duplicated(subset=['group'], keep='first')]

In [ ]:
#Remove near duplicates
result = result[~result[input_id_field].isin(fuzzy_docs_to_remove[input_id_field])]

#Save final result to local
result.to_parquet(dudped_output_dir, write_to_filename=True)

Verify the result of duplicate removal. We can see that the number of document in resultant document is less than the original dataset 

In [ ]:
res = pd.read_parquet(dudped_output_dir)
print(f"Length of duplicate removed dataset:{len(res)}")

Close the GPU Dask Cluster.You might encounter error such as `Caught signal 11`.It's OK, just rerun the cell again.

In [ ]:
client.cluster.close()
client.shutdown()

## 7. Heuristic Fitlering

In this section, we will apply multiple heuristic filters to the dataset, record the heuristic score for documents and documents removed for each filter. For each heuristic filter, the filter calculates a quality scores based on user defined heuristics/algorithms and classifies documents into high quality documents or low quality documents if the quality score is above the user defined threshold.

Sample lists of heuristic filters can be found in `./config/`
- `heuristic_filter_en.yaml`: Sample heuristic filter list for English dataset
- `heuristic_filter_non-en.yaml`:Sample heuristic filter list for Non-English dataset
- `heuristic_filter_code.yaml`:Sample heuristic filter list for Code language dataset
Please adjust the sample list e.g. remove/add filters or change filter threshold based on your own use case. In this example, `heuristic_filter_non-en.yaml` will be used.

For detailed implementation and description of each heuristic filter, please refer to `./NeMo-Curator/nemo-curator/filters/heuristics_filter.py`. For customized heuristic filter implementation, user shall follow the sample implementations, write customized filters and update the .yaml files accordingly.

For analysis of impact of each filters on the dataset, user should set `log-score` to true for the filters in the corresponding config .yaml file. This will output quality score for all filters in separate .txt files for each individual filter. With the quality score and filter threshold, use can calculate quality score distribution and other analysis to assess the effectiveness of each filter.

In this example, in order to get a comprehensive output of each filter, we are iterating through ever filter using a for loop and saving the intermediate result. This process will involve extensive I/O operations and is less effective. Alternatively, after loading input dataset and filter pipeline, user can simply call `filter_pipeline(dataset)` to obtain the final filtered result.

In [ ]:
from nemo_curator.utils.config_utils import build_filter_pipeline
from nemo_curator import Score, Filter, ScoreFilter
from nemo_curator.utils.file_utils import get_batched_files,expand_outdir_and_mkdir

**[Optional]** The following cell is to remove warning from dask.

In [ ]:
import warnings

# Disable the metadata warning
warnings.filterwarnings("ignore",module="dask.dataframe.core")

Create a CPU Dask Cluster.

In [ ]:
cluster = LocalCluster(n_workers=10, processes=True, memory_limit='16GB')
client = Client(cluster)

2024-08-12 06:36:51,616 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:32917' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('getitem-5b69d236ac9974e9fb86010ffc64382a', 0), ('getitem-1a1421e1fc0bebcfdb81496a35f59d59', 0), ('getitem-a531838794cbb6793b5455275c088d56', 0), ('getitem-5a479f5a8ba45819d7bc110e6f66c5cf', 0), ('getitem-20cb1fb330d399835eab7d541c90d9ad', 0), ('getitem-ea8820d11bd559a47001726946b401f1', 0), ('getitem-dc3a1400f3d825aa608fea3f19009402', 0), ('getitem-fc7ee0a305222d3cbc86116635f8f1b7', 0), ('getitem-5a35ddcf8be5c285f2cc9e07ba4168d6', 0), ('getitem-9f6e0b039afa9a3a892b2eee42fff9ff', 0), ('getitem-aef58cc24b78e9deb456d9854d8056db', 0), ('getitem-cf46f299cd36329b1ec712d5fd751b3a', 0), ('getitem-36157dd00770b4907cf863f121981541', 0), ('getitem-2d4c129c73f6e4bd0add5175ea806475', 0)} (stimulus_id='handle-worker-cleanup-1723444611.6157267')


Define some helper functions

In [ ]:
def get_dataframe_complement(original_df, filtered_df):
    def partition_complement(part_original_df, partition_info=None):
        if not partition_info:
            return part_original_df
        part_filtered_df = filtered_df.get_partition(partition_info["number"])
        complement_mask = ~part_original_df.index.isin(part_filtered_df.index.persist())
        complement_df = part_original_df[complement_mask]
        return complement_df

    return original_df.map_partitions(partition_complement)

def write_scores(df, output_dir):
    for column in df.columns:
        output_path = os.path.join(output_dir, f"{column}.txt")
        df[column].to_csv(output_path, single_file=True, encoding="utf-8", header=False, index=False, mode="a")

def get_score_fields(pipeline):
    score_fields = []
    for nc_module in pipeline.modules:
        if isinstance(nc_module, Score) or isinstance(nc_module, ScoreFilter):
            if nc_module.score_field:
                score_fields.append(nc_module.score_field)
    return score_fields

Define parameters

In [ ]:
#Input
HF_input_data_dir = dudped_output_dir
input_file_type = 'parquet'
batch_size = 1

#Output
HF_base_output_path = os.path.join(data_dir,'heuristic_filtering')
kept_document_dir =  os.path.join(HF_base_output_path,'data','hq.parquet')
removed_document_dir =  os.path.join(HF_base_output_path,'data','lq.parquet')
output_document_score_dir =  os.path.join(HF_base_output_path,'data','score')
output_file_type = 'parquet'

#Relevant parameters
filter_config_file = './config/heuristic_filter_non-en.yaml'
input_id_field = 'id'

#Set to False if do not want to save intermediate results
is_cache = True

!mkdir -p {kept_document_dir}
!mkdir -p {removed_document_dir}
!mkdir -p {output_document_score_dir}

Run heuristic filtering

In [ ]:
t0 = time.time()

#Load filters from config
filter_pipeline = build_filter_pipeline(filter_config_file)
score_fields = get_score_fields(filter_pipeline)

# Load dataset
dataset = DocumentDataset.read_parquet(HF_input_data_dir, backend='pandas', add_filename=True)


# Iterate through filters. For each filter, the low quality document will be removed from the dataset and output to corresponding folder for analysis
# Output of previous filter will be input of the next filter
if is_cache:
    curr_dataset = prev_dataset = dataset
    for filter_module in filter_pipeline.modules:
        #Apply filter
        curr_dataset = filter_module(curr_dataset).persist()

        #Output filtered document
        print(f"Saving data for {filter_module.filter_obj._name}")
        removed_df = get_dataframe_complement(prev_dataset.df, curr_dataset.df)
        removed_filter_dir = os.path.join(removed_document_dir, filter_module.filter_obj._name)
        expand_outdir_and_mkdir(removed_filter_dir)
        write_to_disk(removed_df, removed_filter_dir, write_to_filename=True, output_type=output_file_type)
        prev_dataset = curr_dataset
    filtered_dataset = curr_dataset
else:
    filtered_dataset = filter_pipeline(dataset)

# Write scores of retained doucment to separate directory
output_df = filtered_dataset.df[[input_id_field, *score_fields]]
write_scores(output_df, output_document_score_dir)

# Remove scores from dataset df
filtered_dataset = DocumentDataset(filtered_dataset.df.drop(columns=score_fields))

# Output filtered dataset
filtered_dataset.to_parquet(kept_document_dir, write_to_filename=True)

print(f"Time taken for Heuristic filtering: {time.time()-t0} s")

**[Optional]** Verify the result.

In [ ]:
# res = pd.read_parquet(kept_document_dir)
# print(f"Dataset size after heuristic filtering:{len(res)}")
# res.head()

Close the CPU Dask Cluster

In [64]:
client.cluster.close()
client.shutdown()